# Neural Network Binary Classification

## **1 Introduction**

This notebook is my learning material to keep track of the notions approached in the [Advanced Learning Algorithms](https://www.coursera.org/learn/advanced-learning-algorithms?specialization=machine-learning-introduction) course from the [Machine Learning Specialization](https://www.coursera.org/specializations/machine-learning-introduction) offered by DeepLearning.AI and Standford University.

Through this notebook, I use the [Heart Failure Prediction Dataset](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction) created by fedesoriano.

### **1.0.1 Imports**

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Machine Learning
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Options for seaborn
sns.set_style('darkgrid')
%matplotlib inline

from IPython import get_ipython
ipython = get_ipython()

# Autoreload extesnions
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

### **1.1 Data**

#### **1.1.0.1 Import**

In [ ]:
heart = pd.read_csv('heart.csv')
heart

#### **1.1.1 Exploratory Data Analysis**

In [ ]:
heart.info()
heart.describe()

In [ ]:
sns.pairplot(data=heart, hue='HeartDisease',
             height=1.5)

In [ ]:
sns.heatmap(data=heart.corr(),
            cmap='coolwarm',
            annot=True)

## **2 TensorFlow implementation**

### **2.1 Data preparation**

#### **2.1.1 Label encoding**

In [ ]:
le = LabelEncoder()
for col in heart.select_dtypes(include='object').columns:
    heart[col] = le.fit_transform(heart[col])
    
heart

#### **2.1.2 Splitting data**

In [ ]:
training = heart.sample(frac=0.8, random_state=12)
testing = heart.drop(training.index)

X_train = training.drop('HeartDisease', axis=1).to_numpy()
y_train = training['HeartDisease'].values

X_test = testing.drop('HeartDisease', axis=1).to_numpy()
y_test = testing['HeartDisease'].values

#### **2.1.3 Feature scaling**

In [ ]:
ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

### **2.2 Analysis**

#### **2.2.1 Model**

In [ ]:
model = Sequential([
    tf.keras.Input(shape=(11,)),
    Dense(units=10, activation='relu'),
    Dense(units=5, activation='relu'),
    Dense(units=1, activation='sigmoid')
], name='heart_failure_detection')

model.summary()

model.compile(loss=BinaryCrossentropy(),
              optimizer=Adam(1e-3))

history = model.fit(X_train, y_train,
                    epochs=250)

#### **2.2.2 Predict**

In [ ]:
predictions = model.predict(X_test)
predictions

### **2.3 Results**

#### **2.3.1 Accuracy**

In [ ]:
tf.metrics.binary_accuracy(y_test, predictions.flatten(),
                           threshold=0.5)

#### **2.3.2 Loss evolution**

In [ ]:
sns.lineplot(x=history.epoch, y=history.history['loss'])

plt.xlabel('epoch')
plt.ylabel('loss')

## **3 NumPy implementation (forward propagation)**

### **3.1 Analysis**

#### **3.1.1 Activation function**

$$
sigmoid(z) = \frac{1}{1 + e^{-z}}
$$

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

#### **3.1.2 Dense layer**

$$
\vec{a_{out}} = g(\vec{a_{in}} \times \vec{w} + b)
$$

In [ ]:
def dense_layer(A_in, W, b, g):
    return g(A_in @ W + b)

#### **3.1.3 Neural Network**

In [ ]:
def sequential(X, W1, b1, W2, b2, W3, b3):
    A1 = dense_layer(X, W1, b1, sigmoid)
    A2 = dense_layer(A1, W2, b2, sigmoid)
    A3 = dense_layer(A2, W3, b3, sigmoid)
    
    return A3

#### **3.1.4 Predict**

In [ ]:
# Retreive weights and biases from previous model
layer1, layer2, layer3 = model.layers

W1, b1 = layer1.get_weights()
W2, b2 = layer2.get_weights()
W3, b3 = layer3.get_weights()

predictions = sequential(X_test, W1, b1, W2, b2, W3, b3)
predictions

### **3.2 Results**

#### **3.2.1 Accuracy**

In [ ]:
tf.metrics.binary_accuracy(y_test, predictions.flatten(),
                           threshold=0.5)